# 收益管理

## 目标和前提条件

在这个例子中，我们将展示数学优化如何让您的收入和利润飙升，我们将向您展示航空公司如何使用人工智能技术制定最佳座位定价策略。您将学习如何使用Gurobi Python API将此收益管理问题建模为三期随机规划问题并使用Gurobi优化器求解。

此模型是H. Paul Williams所著《Model Building in Mathematical Programming》第五版第282-284页和337-340页的示例24。

这个建模示例属于高级水平，我们假设您了解Python和Gurobi Python API，并且您具有构建数学优化模型的高级知识。通常，这些示例的目标函数和/或约束很复杂，或者需要Gurobi Python API的高级功能。

**下载仓库** <br /> 
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的仓库。

**Gurobi许可证** <br /> 
为了正确运行此Jupyter笔记本，您必须拥有Gurobi许可证。如果您没有，您可以作为*商业用户*申请[评估许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/)，或者作为*学术用户*下载[免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses/)。

## 问题描述

一家航空公司正在销售前往特定目的地的航班机票。航班将在三周后起飞，最多需要六架飞机，每架飞机的租用成本为50,000英镑。每架飞机有以下座位：

* 37个头等舱座位
* 38个商务舱座位
* 47个经济舱座位

航空公司需要决定这些座位的初始价格，然后在一周和两周后有机会更新价格。一旦客户购买了机票，就没有取消选项。为了简化管理，每个舱位都有三种可能的价格水平选项（必须选择其中一种）。每个舱位不必选择相同的选项。下表给出了当前期间（第1期）和未来两个期间的价格选项。

![optionsClass](optionsClass.PNG)

需求是不确定的，但会受到价格的影响。根据将需求水平分为每个期间三种情景的概率分布进行了需求预测。三种情景在每个期间的概率如下：

![scenariosProb](scenariosProb.PNG)

预测的需求水平如下表所示：

![forecastDemand](forecastDemand.PNG)

目标是确定当前期间的价格水平，在每个舱位销售多少座位，预订的飞机数量，以及未来期间的临时价格水平和销售座位数，以最大化预期收益。我们应该能够在所有可能的情景组合下满足承诺。事后（即直到下一期开始才知道），每个期间的实际需求（取决于您设定的价格水平）如下表所示。

![actualDemand](actualDemand.PNG)

我们使用第1期设定价格后产生的实际需求，在第2期开始时重新运行模型，以设定第2期的价格水平和第3期的临时价格水平。我们在第3期开始时重复这一过程进行重新运行。

---
## 模型公式

收益管理问题被建模为三期随机规划问题。首次求解此模型给出第1周的价格水平和销售建议，并在所有可能的情景下推荐后续周的价格水平和销售。这些情景的概率将被考虑，以最大化预期收益。一周后，模型将重新运行，考虑第一周的已承诺销售和收入，重新确定第二周（即"追索"）和第三周在所有可能情景下的推荐价格和销售。该程序将在一周后再次重复。

### 集合和索引

$i,j,k \in \text{Scenarios}$：情景的索引和集合。

$h \in \text{Options}$：价格选项的索引和集合。

$c \in \text{Class}$：座位类别的索引和集合。

### 参数

$\text{price1}_{c,h} \in \mathbb{R}^+$：第1周为类别$c$选择的选项$h$的价格。

$\text{price2}_{i,c,h} \in \mathbb{R}^+$：由于第1周情景$i$导致的第2周为类别$c$选择的选项$h$的价格。

$\text{price3}_{i,j,c,h} \in \mathbb{R}^+$：由于第1周情景$i$和第2周情景$j$导致的第3周为类别$c$选择的选项$h$的价格。

$\text{forecast1}_{i,c,h} \in \mathbb{R}^+$：第1周在价格选项$h$和情景$i$下类别$c$的预测需求。

$\text{forecast2}_{i,j,c,h} \in \mathbb{R}^+$：如果第1周持有情景$i$，第2周持有情景$j$，则第2周在价格选项$h$下类别$c$的预测需求。

$\text{forecast3}_{i,j,k,c,h} \in \mathbb{R}^+$：如果第1周持有情景$i$，第2周持有情景$j$，第3周持有情景$k$，则第3周在价格选项$h$下类别$c$的预测需求。

$\text{prob}_i \in [0,1]$：情景$i$的概率。

$\text{cap}_c \in \mathbb{N}$：每架飞机类别$c$的容量。

$\text{cost} \in \mathbb{R}^+$：租用一架飞机的成本。


### 决策变量

$p1_{c,h} \in \{0, 1\}$：如果在第1周为类别$c$选择了选项$h$的价格，则此二元变量等于1。

$p2_{i,c,h} \in \{0, 1\}$：如果由于第1周情景$i$导致在第2周为类别$c$选择了选项$h$的价格，则此二元变量等于1。

$p3_{i,j,c,h} \in \{0, 1\}$：如果由于第1周情景$i$和第2周情景$j$导致在第3周为类别$c$选择了选项$h$的价格，则此二元变量等于1。

$s1_{i,c,h} \in \mathbb{R}^+$：在情景$i$下，按价格选项$h$在第1周为类别$c$销售的机票数量。

$s2_{i,j,c,h} \in \mathbb{R}^+$：如果第1周持有情景$i$，第2周持有情景$j$，则按价格选项$h$在第2周为类别$c$销售的机票数量。

$s3_{i,j,k,c,h} \in \mathbb{R}^+$：如果第1周持有情景$i$，第2周持有情景$j$，第3周持有情景$k$，则按价格选项$h$在第3周为类别$c$销售的机票数量。

$n \in \mathbb{N}$：要飞行的飞机数量。

### 目标函数

**利润**：最大化预期利润。

$$
\text{最大化} \quad \text{profit} = (
\sum_{i \in \text{Scenarios}} \sum_{c \in \text{Class}} 
\sum_{h \in \text{Options}}{\text{prob}_i * \text{price1}_{c,h} * p1_{c,h} * s1_{i,c,h}  }  +
$$

$$
\sum_{i \in \text{Scenarios}} \sum_{j \in \text{Scenarios}} \sum_{c \in \text{Class}} \sum_{h \in \text{Options}}
{\text{prob}_i *\text{prob}_j * \text{price2}_{c,h} * p2_{i,c,h} * s2_{i,j,c,h} } +
$$

$$
\sum_{i \in \text{Scenarios}} \sum_{j \in \text{Scenarios}} \sum_{k \in \text{Scenarios}} 
\sum_{c \in \text{Class}} \sum_{h \in \text{Options}}
{\text{prob}_i * \text{prob}_j * \text{prob}_k * \text{price3}_{c,h} * p2_{i,j,c,h} * s3_{i,j,k,c,h}} )
- \text{cost} * n
$$

### 约束条件

**第1周价格选项**：在第1周，每个舱位必须只选择一个价格选项。

$$
\sum_{h \in \text{Options}} p1_{c,h} = 1 \quad \forall c \in \text{Class}
$$

**第1周销售**：第1周的销售不能超过预测需求。

$$
s1_{i,c,h} \leq \text{forecast1}_{i,c,h} * p1_{c,h},
\quad \forall i \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
$$

**第2周价格选项**：对于第1周的每个情景，在第2周的每个舱位必须只选择一个价格选项。

$$
\sum_{h \in \text{Options}} p2_{i,c,h} = 1 \quad \forall c \in \text{Class}, \; i \in \text{Scenarios}
$$

**第2周销售**：第2周的销售不能超过预测需求。

$$
s2_{i,j,c,h} \leq \text{forecast2}_{j,c,h} * p2_{i,c,h},
\quad \forall i,j \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
$$

**第3周价格选项**：对于第1周和第2周的每个情景，在第3周的每个舱位必须只选择一个价格选项。

$$
\sum_{h \in \text{Options}} p3_{i,j,c,h} = 1 \quad \forall c \in \text{Class}, \; i,j \in \text{Scenarios}
$$

**第3周销售**：第3周的销售不能超过预测需求。
$$
s3_{i,j,k,c,h} \leq \text{forecast3}_{k,c,h} * p3_{i,j,c,h},
\quad \forall i,j,k \in \text{Scenarios}, \; c \in \text{Class}, \; h \in \text{Options}
$$

**舱位容量**：每个舱位的容量约束。

$$
\sum_{h \in \text{Options}} s1_{i,c,h} +
\sum_{h \in \text{Options}} s2_{i,j,c,h} + 
\sum_{h \in \text{Options}} s3_{i,j,k,c,h} \leq \text{cap}_c * n
\quad \forall i,j,k \in \text{Scenarios}, \; c \in \text{Class}
$$

**飞机数量** 最多可以租用六架飞机。

$$
n \leq 6
$$

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
%pip install gurobipy
# 注意，受限制的许可证不足以运行此笔记本，需要完整许可证

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# 使用Python 3.7.0和Gurobi 9.1.0测试

## 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 舱位、价格选项和情景的列表。

classes = ['First', 'Business', 'Economy']

options = ['option1', 'option2', 'option3']

scenarios = ['sce1', 'sce2', 'sce3']

# 第1、2和3周的舱位、价格选项和价格值。

ch, price1, price2, price3  = gp.multidict({
    ('First', 'option1'): [1200, 1400, 1500],
    ('Business', 'option1'): [900, 1100, 820],
    ('Economy', 'option1'): [500, 700, 480],
    ('First', 'option2'): [1000, 1300, 900],
    ('Business', 'option2'): [800, 900, 800],
    ('Economy', 'option2'): [300, 400, 470],
    ('First', 'option3'): [950, 1150, 850],
    ('Business', 'option3'): [600, 750, 500],
    ('Economy', 'option3'): [200, 350, 450]
})

# 每个情景的概率

prob ={'sce1': 0.1, 'sce2': 0.7, 'sce3': 0.2}

# 第1周每个舱位、价格选项和情景的预测需求

ich, fcst1, fcst2, fcst3 = gp.multidict({
    ('sce1', 'First', 'option1'): [10, 20, 30],
    ('sce1', 'Business', 'option1'): [20, 42, 40],
    ('sce1', 'Economy', 'option1'): [45, 50, 50],
    ('sce1', 'First', 'option2'): [15, 25, 35],
    ('sce1', 'Business', 'option2'): [25, 45, 50],
    ('sce1', 'Economy', 'option2'): [55, 52, 60],
    ('sce1', 'First', 'option3'): [20, 35, 40],
    ('sce1', 'Business', 'option3'): [35, 46, 55],
    ('sce1', 'Economy', 'option3'): [60, 60, 80],
    ('sce2', 'First', 'option1'): [20, 10, 30],
    ('sce2', 'Business', 'option1'): [40, 50, 10],
    ('sce2', 'Economy', 'option1'): [50, 60, 50],
    ('sce2', 'First', 'option2'): [25, 40, 40],
    ('sce2', 'Business', 'option2'): [42, 60, 40],
    ('sce2', 'Economy', 'option2'): [52, 65, 60],
    ('sce2', 'First', 'option3'): [35, 50, 60],
    ('sce2', 'Business', 'option3'): [45, 80, 45],
    ('sce2', 'Economy', 'option3'): [63, 90, 70],
    ('sce3', 'First', 'option1'): [45, 50, 50],
    ('sce3', 'Business', 'option1'): [45, 20, 40],
    ('sce3', 'Economy', 'option1'): [55, 10, 60],
    ('sce3', 'First', 'option2'): [50, 55, 70],
    ('sce3', 'Business', 'option2'): [46, 30, 45],
    ('sce3', 'Economy', 'option2'): [56, 40, 65],
    ('sce3', 'First', 'option3'): [60, 80, 80],
    ('sce3', 'Business', 'option3'): [47, 50, 60],
    ('sce3', 'Economy', 'option3'): [64, 60, 70]    
})

# 第1、2和3周的实际需求。

ch, demand1, demand2, demand3  = gp.multidict({
    ('First', 'option1'): [25, 22, 45],
    ('Business', 'option1'): [50, 45, 20],
    ('Economy', 'option1'): [50, 50, 55],
    ('First', 'option2'): [30, 45, 60],
    ('Business', 'option2'): [40, 55, 40],
    ('Economy', 'option2'): [53, 60, 60],
    ('First', 'option3'): [40, 50, 75],
    ('Business', 'option3'): [45, 75, 50],
    ('Economy', 'option3'): [65, 80, 75]
})

# 舱位容量

cap ={'First': 37, 'Business': 38, 'Economy': 47}

# 每架飞机的成本

cost = 50000

In [ ]:
# 预处理

# 第1周数据结构
list_ch = []

for c in classes:
    for h in options:
        tp = c,h
        list_ch.append(tp)

ch = gp.tuplelist(list_ch)

list_ich = []

for i in scenarios:
    for c in classes:
        for h in options:
            tp = i,c,h
            list_ich.append(tp)

ich = gp.tuplelist(list_ich)

# 第2周数据结构

list_ijch = []

for i in scenarios:
    for j in scenarios:
        for c in classes:
            for h in options:
                tp = i,j,c,h
                list_ijch.append(tp)

ijch = gp.tuplelist(list_ijch)

# 第3周数据结构

list_ijkch = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            for c in classes:
                for h in options:
                    tp = i,j,k,c,h
                    list_ijkch.append(tp)

ijkch = gp.tuplelist(list_ijkch)

# 情景数据结构

list_ijk = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            tp = i,j,k,
            list_ijk.append(tp)

ijk = gp.tuplelist(list_ijk)

# 容量约束数据结构

list_ijkc = []

for i in scenarios:
    for j in scenarios:
        for k in scenarios:
            for c in classes:
                tp = i,j,k,c
                list_ijkc.append(tp)

ijkc = gp.tuplelist(list_ijkc)

## 模型部署

使用Gurobi求解双线性问题就像配置全局参数`nonConvex`并将此参数设置为值2一样简单。

### 第一期模型

在第1周开始时，我们想确定本周的价格选项。

我们创建一个模型和变量。

In [ ]:
model = gp.Model('YieldManagement')

# 设置全局参数
model.params.nonConvex = 2

# 决策变量

# 每周的价格选项二元变量
p1ch = model.addVars(ch, vtype=GRB.BINARY, name="p1ch")
p2ich = model.addVars(ich, vtype=GRB.BINARY, name="p2ich")
p3ijch = model.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# 每周要销售的机票
s1ich = model.addVars(ich, name="s1ich")
s2ijch = model.addVars(ijch, name="s2ijch")
s3ijkch = model.addVars(ijkch, name="s3ijkch")

# 要飞行的飞机数量
n = model.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

Using license file c:\gurobi\gurobi.lic
Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


### 第1周约束

以下约束确保在第1周的每个舱位中只选择一个价格选项。

In [ ]:
# 第1周的价格选项约束

priceOption1 = model.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

以下约束强制销售不能超过第1周的预测需求。

In [ ]:
# 第1周的销售约束

sales1 = model.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

### 第2周约束

对于第1周的每个情景，在第2周的每个舱位中必须只选择一个价格选项。

In [ ]:
# 第2周的价格选项约束

priceOption2 = model.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

销售不能超过第2周的预测需求。

In [ ]:
# 第2周的销售约束

sales2 = model.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

### 第3周约束

对于第1周和第2周的每个情景，在第3周的每个舱位中必须只选择一个价格选项。

In [ ]:
# 第3周的价格选项约束

priceOption3 = model.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

销售不能超过第3周的预测需求。

In [ ]:
# 第3周的销售约束

sales3 = model.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

每个舱位的容量约束。

In [ ]:
# 舱位容量约束

classCap = model.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

目标是最大化预期利润。

In [ ]:
# 目标函数
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model.setObjective( obj, GRB.MAXIMIZE)

In [ ]:
# 验证模型公式

model.write('YieldManagement.lp')

# 运行优化引擎

model.optimize()

#############################################################
#            打印第1周模型的结果
#############################################################

print("\n\n\n____________________第1周解决方案___________________________")

print(f"预期总利润为：£{round(model.objVal,2): ,}") 
print(f"预订的飞机数量：{n.x}")

# 第1周各舱位的价格

# 第1周选项价格的最优值
opt_p1ch = {}

print("\n____________________第1周价格_______________________________")
for c,h in ch:
    opt_p1ch[c,h] = 0
    if p1ch[c,h].x > 0.5:
        opt_p1ch[c,h] = round(p1ch[c,h].x)
        price_ch = opt_p1ch[c,h]*price1[c,h]
        print(f"({c},{h}) = £{price_ch: ,}")
#

# 第2周临时价格

print("\n_____________第2周临时价格____________________________")
for i,c,h in ich:
    if p2ich[i,c,h].x > 0.5:
        price_ch = round(p2ich[i,c,h].x)*price2[c,h]
        print(f"({i}, {c}, {h}) = £{price_ch: ,}")

# 第3周临时价格

print("\n_____________第3周临时价格____________________________")
for i,j,c,h in ijch:
    if p3ijch[i,j,c,h].x > 0.5:
        price_ch = round(p3ijch[i,j,c,h].x)*price3[c,h]
        print(f"({i}, {j}, {c}, {h}) = £{price_ch: ,}")

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1629 nonzeros
Model fingerprint: 0x21b1ab1b
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -300000.0000
Presolve time: 0.00s
Presolved: 822 rows, 820 columns, 2682 nonzeros
Variable types: 702 continuous, 118 integer (117 binary)

Root relaxation: objective -1.704970e+05, 487 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 170497.004    0   15 -300000.00 170497.004   157%     -    0s
H    0     0

### 第二期模型

我们使用第1周的实际需求运行模型，并确定第2周的价格选项。

In [ ]:
model2 = gp.Model('YieldManagement2')

# 设置全局参数
model2.params.nonConvex = 2

# 决策变量

# 每周的价格选项二元变量
p1ch = model2.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# 固定第1周的价格选项
for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model2.addVars(ich, vtype=GRB.BINARY, name="p2ich")
p3ijch = model2.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# 每周要销售的机票
s1ich = model2.addVars(ich, name="s1ich")

# 使用第1周的事后需求
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h] 

s2ijch = model2.addVars(ijch, name="s2ijch")
s3ijkch = model2.addVars(ijkch, name="s3ijkch")

# 要飞行的飞机数量
n = model2.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# 第1周的价格选项约束

priceOption1 = model2.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# 第1周的销售约束

sales1 = model2.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# 第2周的价格选项约束

priceOption2 = model2.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# 第2周的销售约束

sales2 = model2.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# 第3周的价格选项约束

priceOption3 = model2.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# 第3周的销售约束

sales3 = model2.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# 舱位容量约束

classCap = model2.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# 目标函数
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model2.setObjective( obj, GRB.MAXIMIZE)

# 验证模型公式

model2.write('YieldManagement2.lp')

# 运行优化引擎

model2.optimize()

#############################################################
#            打印第2周模型的结果
#############################################################

print("\n\n\n____________________第2周解决方案___________________________")

print(f"第2周开始时的预期总利润为：£ {round(model2.objVal,2):,}") 
print(f"预订的飞机数量：{n.x}")

# 第2周价格

# 第1周选项价格的最优值
opt_p2ich = {}

print("\n_____________第2周价格____________________________")
for i,c,h in ich:
    opt_p2ich[i,c,h] = 0
    if p2ich[i,c,h].x > 0.5:
        opt_p2ich[i,c,h] = round(p2ich[i,c,h].x)
        price_ch = opt_p2ich[i,c,h]*price2[c,h]
        #print(f"({i},{c},{h}) = £ {price_ch}")
        if i == 'sce1':
            print(f"({c},{h}) = £{price_ch: ,}")
#

# 第3周临时价格

print("\n_____________第3周临时价格____________________________")
for i,j,c,h in ijch:
    if p3ijch[i,j,c,h].x > 0.5:
        price_ch = round(p3ijch[i,j,c,h].x)*price3[c,h]
        print(f"({i}, {j}, {c}, {h}) = £ {price_ch}")

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1611 nonzeros
Model fingerprint: 0x92c92b4e
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -300000.0000
Presolve removed 30 rows and 27 columns
Presolve time: 0.00s
Presolved: 765 rows, 766 columns, 2376 nonzeros
Variable types: 657 continuous, 109 integer (108 binary)

Root relaxation: objective -1.743903e+05, 456 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

# 第三期模型
我们使用第1周和第2周的实际需求运行模型。

In [ ]:
model3 = gp.Model('YieldManagement3')

# 设置全局参数
model3.params.nonConvex = 2

# 决策变量

# 每周的价格选项二元变量
p1ch = model3.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# 固定第1周的价格选项

for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model3.addVars(ich, vtype=GRB.BINARY, name="p2ich")

# 固定第2周的价格选项

for i,c,h in ich:
    p2ich[i,c,h].lb = opt_p2ich[i,c,h]

p3ijch = model3.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# 每周要销售的机票
s1ich = model3.addVars(ich, name="s1ich")

# 使用第1周的事后需求
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h]

s2ijch = model3.addVars(ijch, name="s2ijch")

# 使用第2周的事后需求
for j,c,h in ich:
    fcst2[j,c,h] = 0
    fcst2[j,c,h] = demand2[c,h]*opt_p2ich[j,c,h]

s3ijkch = model3.addVars(ijkch, name="s3ijkch")

# 要飞行的飞机数量
n = model3.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# 第1周的价格选项约束

priceOption1 = model3.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# 第1周的销售约束

sales1 = model3.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# 第2周的价格选项约束

priceOption2 = model3.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# 第2周的销售约束

sales2 = model3.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# 第3周的价格选项约束

priceOption3 = model3.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# 第3周的销售约束

sales3 = model3.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# 舱位容量约束

classCap = model3.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# 目标函数
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model3.setObjective( obj, GRB.MAXIMIZE)

# 验证模型公式

model3.write('YieldManagement3.lp')

# 运行优化引擎

model3.optimize()

#############################################################
#            打印第3周模型的结果
#############################################################


print("\n\n\n____________________第3周解决方案___________________________")

print(f"预期总利润为：£ {round(model3.objVal,2):,}") 
print(f"预订的飞机数量：{n.x}")

# 第3周价格

# 第3周选项价格的最优值
opt_p3ijch = {}


print("\n_____________第3周价格____________________________")
for i,j,c,h in ijch:
    opt_p3ijch[i,j,c,h] = 0
    if p3ijch[i,j,c,h].x > 0.5:
        opt_p3ijch[i,j,c,h] = round(p3ijch[i,j,c,h].x)
        price_ch = opt_p3ijch[i,j,c,h]*price3[c,h]
        if i == 'sce1' and j == 'sce1':
            print(f"({c}, {h}) = £{price_ch: ,}")

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1557 nonzeros
Model fingerprint: 0x9217fd30
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -300000.0000
Presolve removed 120 rows and 108 columns
Presolve time: 0.00s
Presolved: 594 rows, 604 columns, 1782 nonzeros
Variable types: 522 continuous, 82 integer (81 binary)

Root relaxation: objective -1.786023e+05, 389 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

# 起飞时的解决方案
我们使用第1、2和3周的实际需求运行模型。

In [ ]:
model4 = gp.Model('YieldManagement4')

# 设置全局参数
model4.params.nonConvex = 2

# 决策变量

# 每周的价格选项二元变量
p1ch = model4.addVars(ch, vtype=GRB.BINARY, name="p1ch")

# 固定第1周的价格选项

for c,h in ch:
    p1ch[c,h].lb = opt_p1ch[c,h]

p2ich = model4.addVars(ich, vtype=GRB.BINARY, name="p2ich")

# 固定第2周的价格选项

for i,c,h in ich:
    p2ich[i,c,h].lb = opt_p2ich[i,c,h]

p3ijch = model4.addVars(ijch, vtype=GRB.BINARY, name="p3ijch")

# 捕获一个opt_p3ijch[i,j,c,h] = 1的情景
opt_p3kch = {}

for i,j,c,h in ijch:
    p3ijch[i,j,c,h].lb = opt_p3ijch[i,j,c,h]
    opt_p3kch[j,c,h] = 0
    if opt_p3ijch[i,j,c,h] == 1:
        opt_p3kch[j,c,h] = opt_p3ijch[i,j,c,h] 

# 每周要销售的机票
s1ich = model4.addVars(ich, name="s1ich")

# 使用第1周的事后需求
for i,c,h in ich:
    fcst1[i,c,h] = 0
    fcst1[i,c,h] = demand1[c,h]*opt_p1ch[c,h] 

s2ijch = model4.addVars(ijch, name="s2ijch")

# 使用第2周的事后需求
for j,c,h in ich:
    fcst2[j,c,h] = 0
    fcst2[j,c,h] = demand2[c,h]*opt_p2ich[j,c,h]


s3ijkch = model4.addVars(ijkch, name="s3ijkch")

# 使用第3周的事后需求
for k,c,h in ich:
    fcst3[k,c,h] = 0
    fcst3[k,c,h] = demand3[c,h]*opt_p3kch[k,c,h]

    
# 要飞行的飞机数量
n = model4.addVar(ub=6, vtype=GRB.INTEGER, name="planes")

# 第1周的价格选项约束

priceOption1 = model4.addConstrs( (gp.quicksum(p1ch[c,h] for h in options ) == 1 for c in classes ), name='priceOption1' )

# 第1周的销售约束

sales1 = model4.addConstrs( (s1ich[i,c,h] <= fcst1[i,c,h]*p1ch[c,h] for i,c,h in ich ), name='sales1' )

# 第2周的价格选项约束

priceOption2 = model4.addConstrs( (gp.quicksum(p2ich[i,c,h] for h in options ) 
                                  == 1 for i in scenarios for c in classes ), name='priceOption2' )

# 第2周的销售约束

sales2 = model4.addConstrs( (s2ijch[i,j,c,h] <= fcst2[j,c,h]*p2ich[i,c,h] for i,j,c,h in ijch ), name='sales2' )

# 第3周的价格选项约束

priceOption3 = model4.addConstrs( (gp.quicksum(p3ijch[i,j,c,h] for h in options ) 
                                  == 1 for i in scenarios for j in scenarios for c in classes ), name='priceOption3' )

# 第3周的销售约束

sales3 = model4.addConstrs( (s3ijkch[i,j,k,c,h] <= fcst3[k,c,h]*p3ijch[i,j,c,h] for i,j,k,c,h in ijkch ), name='sales3' )

# 舱位容量约束

classCap = model4.addConstrs( (gp.quicksum(s1ich[i,c,h] for h in options)  
                              + gp.quicksum(s2ijch[i,j,c,h] for h in options) 
                              + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)  
                              <= cap[c]*n for i,j,k,c in ijkc ) , name='classCap')

# 目标函数
obj = gp.quicksum(prob[i]*price1[c,h]*p1ch[c,h]*s1ich[i,c,h] for i,c,h in ich ) \
+ gp.quicksum(prob[i]*prob[j]*price2[c,h]*p2ich[i,c,h]*s2ijch[i,j,c,h] for i,j,c,h in ijch ) \
+ gp.quicksum(prob[i]*prob[j]*prob[k]*price3[c,h]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h] for i,j,k,c,h in ijkch) - cost*n

model4.setObjective( obj, GRB.MAXIMIZE)

# 验证模型公式

model4.write('YieldManagement4.lp')

# 运行优化引擎

model4.optimize()

#############################################################
#            打印第4周模型的结果
#############################################################


print("\n\n\n____________________起飞解决方案___________________________")

print(f"实际总利润为：£{round(model4.objVal,2):,}") 
print(f"使用的飞机数量：{n.x}")


# 第1周销售

print("\n___________第1周售出座位和收入__________________________")
for i,c,h in ich:
    if i == 'sce1':
        if s1ich[i,c,h].x > 1e-6:
            tickets = round(s1ich[i,c,h].x)
            price = price1[c,h]*round(p1ch[c,h].x)
            revenue = price*tickets
            print(f"{c}舱：以£{price:,}售出{tickets}个座位：收入£{revenue:,}  ")
            
# 第2周销售

print("___________第2期售出座位和收入__________________________")
for i,j,c,h in ijch:
    if i == 'sce1' and j == 'sce1':
        if s2ijch[i,j,c,h].x > 1e-6:
            tickets = round(s2ijch[i,j,c,h].x)
            price = price2[c,h]*round(p2ich[i,c,h].x)
            revenue = price*tickets
            print(f"{c}舱：以£{price:,}售出{tickets}个座位：收入£{revenue:,}  ")
            
# 第3周销售

print("___________第3期售出座位和收入__________________________")
for i,j,k,c,h in ijkch:
    if i == 'sce1' and j == 'sce1' and k == 'sce1':
        if s3ijkch[i,j,k,c,h].x > 1e-6:
            tickets = round(s3ijkch[i,j,k,c,h].x )
            price = price3[c,h]*round(p3ijch[i,j,c,h].x)
            revenue = price*tickets
            print(f"{c}舱：以£{price:,}售出{tickets}个座位：收入£{revenue:,}  ")

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1395 nonzeros
Model fingerprint: 0x0aff6c8c
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 470 rows and 458 columns
Presolve time: 0.00s
Presolved: 1 rows, 11 columns, 11 nonzeros
Variable types: 10 continuous, 1 integer (0 binary)

Root relaxation: objective 1.952617e+05, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

## 参考文献

H. Paul Williams, Model Building in Mathematical Programming, 第五版。

版权所有 © 2020 Gurobi Optimization, LLC